# Read the README [in the repository](https://github.com/jzlotek/cs583-final) before continuing


In [0]:
#
# download_dataset.py
#

import requests
import urllib
import os
import sys
from zipfile import ZipFile

OUTDIR='./dataset'

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True, timeout=None)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True, timeout=None)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 10000
    downloaded = 0
    total = 26926678016.00 * 1.1 / 100.00
    sys.stdout.write("\r0.00% downloaded")
    sys.stdout.flush()

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                downloaded+=len(chunk)
                if downloaded % (CHUNK_SIZE*50) == 0:
                    sys.stdout.write('\r%0.2f%% downloaded' % (downloaded/(total)))
    sys.stdout.write("\r100.00% downloaded")

# Ensure directory exists
if not os.path.exists(OUTDIR):
  os.mkdir(OUTDIR)

filepath = OUTDIR+'/Sony.zip'
# Too much data, download the smaller dataset for now
print('Downloading Sony data... (25GB)')
download_file_from_google_drive('10kpAcvldtcb9G2ze5hTcF1odzu4V_Zvh', filepath)

fileout = OUTDIR+'/Sony'
# Always unzip to reset directory
print('\nUnzipping, this will take a while...')
print('Usage of storage should be around 117GB')
ZipFile(filepath).extractall(path=OUTDIR)

    
print("Data ready in %s" % (fileout))

Data is downloaded, time to train the model.

In [0]:
!pip install keras
!pip install rawpy
!pip install imageio

In [0]:
#
# better training
#

import glob
import numpy as np
import os
import rawpy

from keras import backend as k
from keras.layers import \
Input, Conv2D, LeakyReLU, MaxPooling2D, \
UpSampling2D, Conv2DTranspose, Concatenate, \
ZeroPadding2D, UpSampling3D
from keras.callbacks import LearningRateScheduler
import keras.layers.advanced_activations
import keras.optimizers
from keras.utils import plot_model

import tensorflow as tf

# Constants
target_dir = './dataset/Sony/long/'
input_dir = './dataset/Sony/short/'

ps = 512 # patch size

# Helper Functions
def pack_raw(raw):
    # pack Bayer image to 4 channels
    im = raw.raw_image_visible.astype(np.float32)
    im = np.maximum(im - 512, 0) / (16383 - 512)  # subtract the black level

    im = np.expand_dims(im, axis=2)
    img_shape = im.shape
    H = img_shape[0]
    W = img_shape[1]

    out = np.concatenate((im[0:H:2, 0:W:2, :],
                          im[0:H:2, 1:W:2, :],
                          im[1:H:2, 1:W:2, :],
                          im[1:H:2, 0:W:2, :]), axis=2)
    return out
  
def load_batches(batch_size):   
    train_fns = glob.glob(target_dir + '0*.ARW') 
    train_ids = np.array([int(os.path.basename(train_fn)[0:5]) \
                          for train_fn in train_fns])
    
    train_ids = np.random.permutation(train_ids)
    
    total_length = len(train_ids)
    
    while True:
        batch_start = 0
        batch_end = batch_size

        while batch_start < total_length:
            limit = min(batch_end, total_length)  
            batch_ids = train_ids[batch_start:limit]
            
            # Intialize return values
            inputs, targets = np.empty((batch_size,ps,ps,4)), np.empty((batch_size,ps*2,ps*2,3))
            index = 0
            for image_id in batch_ids:
              
                # Get an random input filename
                input_files = glob.glob(input_dir + '%05d_00*.ARW' % image_id)
                input_path = None
                if len(input_files) <= 1:
                    input_path = input_files[0]
                else :
                    input_path = input_files[np.random.randint(0, len(input_files) - 1)]
                input_fn = os.path.basename(input_path)

                # Get the matching target name
                target_files = glob.glob(target_dir + '%05d_00*.ARW' % image_id)
                target_path = target_files[0]
                target_fn = os.path.basename(target_path)
                          
                # Calculate amplification ratio
                input_exposure = float(input_fn[9:-5])
                target_exposure = float(target_fn[9:-5])
                ratio = min(target_exposure / input_exposure, 300)
                
                # Load image into memory
                raw = rawpy.imread(input_path)
                input_image = \
                  np.expand_dims(pack_raw(raw), axis=0) * ratio

                target_image = rawpy.imread(target_path)
                target_image = target_image.postprocess(use_camera_wb=True, half_size=False, no_auto_bright=True, output_bps=16)
                target_image = np.expand_dims(np.float32(target_image / 65535.0), axis=0)

                # Take a random patch from each image
                H = input_image.shape[1]
                W = input_image.shape[2]

                xx = np.random.randint(0, W - ps)
                yy = np.random.randint(0, H - ps)
                input_patch = input_image[:, yy:yy + ps, xx:xx + ps, :]
                target_patch = target_image[:, yy * 2:yy * 2 + ps * 2, xx * 2:xx * 2 + ps * 2, :]
        
                # Compute random alterations to the images
                if np.random.randint(2, size=1)[0] == 1:  # random flip
                    input_patch = np.flip(input_patch, axis=1)
                    target_patch = np.flip(target_patch, axis=1)
                if np.random.randint(2, size=1)[0] == 1:
                    input_patch = np.flip(input_patch, axis=2)
                    target_patch = np.flip(target_patch, axis=2)
                if np.random.randint(2, size=1)[0] == 1:  # random transpose
                    input_patch = np.transpose(input_patch, (0, 2, 1, 3))
                    target_patch = np.transpose(target_patch, (0, 2, 1, 3))

                input_patch = np.minimum(input_patch, 1.0)

                # Add to return arrays
                inputs[index] = input_patch
                targets[index] = target_patch
                
                del input_patch
                del target_patch
                
                index += 1

            yield (inputs,targets,(batch_start/batch_size))

            del inputs
            del targets
            
            batch_start += batch_size   
            batch_end += batch_size
            
        return

def main():
    # Backend Config
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.gpu_options.per_process_gpu_memory_fraction = 0.5
    k.tensorflow_backend.set_session(tf.Session(config=config))
    k.set_image_data_format('channels_last')
    
    # Define the model
    inputs = Input(shape=(None,None,4,), dtype="float32",
                name="Inputs")

    x = Conv2D(filters=32, kernel_size=(3,3),
            name="conv_1_1", padding='same')(inputs)
    x = LeakyReLU(alpha=0.2, name="conv_1_act_1")(x)
    x = Conv2D(filters=32, kernel_size=(3,3),
            name="conv_1_2", padding='same')(x)
    x1 = LeakyReLU(alpha=0.2, name="conv_1_act_2")(x)
    x = MaxPooling2D(pool_size=(2, 2), padding='same',
            name="conv_1_pool")(x1)

    x = Conv2D(filters=64, kernel_size=(3,3),
            name="conv_2_1", padding='same')(x)
    x = LeakyReLU(alpha=0.2, name="conv_2_act_1")(x)
    x = Conv2D(filters=64, kernel_size=(3,3),
            name="conv_2_2", padding='same')(x)
    x2 = LeakyReLU(alpha=0.2, name="conv_2_act_2")(x)
    x = MaxPooling2D(pool_size=(2, 2), padding='same',
            name="conv_2_pool")(x2)

    x = Conv2D(filters=128, kernel_size=(3,3),
            name="conv_3_1", padding='same')(x)
    x = LeakyReLU(alpha=0.2, name="conv_3_act_1")(x)
    x = Conv2D(filters=128, kernel_size=(3,3),
            name="conv_3_2", padding='same')(x)
    x3 = LeakyReLU(alpha=0.2, name="conv_3_act_2")(x)
    x = MaxPooling2D(pool_size=(2, 2), padding='same',
            name="conv_3_pool")(x3)

    x = Conv2D(filters=256, kernel_size=(3,3),
            name="conv_4_1", padding='same')(x)
    x = LeakyReLU(alpha=0.2, name="conv_4_act_1")(x)
    x = Conv2D(filters=256, kernel_size=(3,3),
            name="conv_4_2", padding='same')(x)
    x4 = LeakyReLU(alpha=0.2, name="conv_4_act_2")(x)
    x = MaxPooling2D(pool_size=(2, 2), padding='same',
            name="conv_4_pool")(x4)

    x = Conv2D(filters=512, kernel_size=(3,3),
            name="conv_5_1", padding='same')(x)
    x = LeakyReLU(alpha=0.2, name="conv_5_act_1")(x)
    x = Conv2D(filters=512, kernel_size=(3,3),
            name="conv_5_2", padding='same')(x)
    x5 = LeakyReLU(alpha=0.2, name="conv_5_act_2")(x)
    
    x5 = Conv2DTranspose(filters=256, kernel_size=(2,2),
            strides=(2,2), padding='same')(x5)
    x = Concatenate(axis=3)([x5,x4])
    x = Conv2D(filters=256, kernel_size=(3,3),
            name="conv_7_1", padding='same')(x)
    x = LeakyReLU(alpha=0.2, name="conv_7_act_1")(x)
    x = Conv2D(filters=256, kernel_size=(3,3),
            name="conv_7_2", padding='same')(x)
    x6 = LeakyReLU(alpha=0.2, name="conv_7_act_2")(x)

    x6 = Conv2DTranspose(filters=128, kernel_size=(2,2),
            strides=(2,2), padding='same')(x6)
    x = Concatenate(axis=3)([x6,x3])
    x = Conv2D(filters=128, kernel_size=(3,3),
            name="conv_8_1", padding='same')(x)
    x = LeakyReLU(alpha=0.2, name="conv_8_act_1")(x)
    x = Conv2D(filters=128, kernel_size=(3,3),
            name="conv_8_2", padding='same')(x)
    x7 = LeakyReLU(alpha=0.2, name="conv_8_act_2")(x)

    x7 = Conv2DTranspose(filters=64, kernel_size=(2,2),
            strides=(2,2), padding='same')(x7)
    x = Concatenate(axis=3)([x7,x2])
    x = Conv2D(filters=64, kernel_size=(3,3),
            name="conv_9_1", padding='same')(x)
    x = LeakyReLU(alpha=0.2, name="conv_9_act_1")(x)
    x = Conv2D(filters=64, kernel_size=(3,3),
            name="conv_9_2", padding='same')(x)
    x8 = LeakyReLU(alpha=0.2, name="conv_9_act_2")(x)

    x8 = Conv2DTranspose(filters=32, kernel_size=(2,2),
            strides=(2,2), padding='same')(x8)
    x = Concatenate(axis=3)([x8,x1])
    x = Conv2D(filters=32, kernel_size=(3,3),
            name="conv_10_1", padding='same')(x)
    x = LeakyReLU(alpha=0.2, name="conv_10_act_1")(x)
    x = Conv2D(filters=32, kernel_size=(3,3),
            name="conv_10_2", padding='same')(x)
    x = LeakyReLU(alpha=0.2, name="conv_10_act_2")(x)

    x = Conv2D(filters=3, kernel_size=(1,1), activation=None,
            name="conv_to_image")(x)

    outputs = UpSampling2D(size=(2,2))(x)

    # Create the model
    model = keras.Model(inputs=inputs, outputs=outputs, name='ltsitd')

    # Get info about the model
    if False:
        model.summary(line_length=150)
        
        plot_model(model, to_file='model.png')
        from google.colab import files
        files.download('model.png')
    
    batch_size = 8          # Number of images to load into memory at once (RAM/GPU Memory restricted)
    total_epochs = 11       # Number of training rounds per batch

    # Drop the learning rate by a factor of 10 halfway through
    def step_decay(epoch):
        lrate = 1e-4
        
        if epoch > total_epochs // 2:
          lrate = 1e-5
        
        return lrate
      
    lrate = LearningRateScheduler(step_decay)
    callback_list = [lrate]
    
    # Build the model
    model.compile(loss='mae',          # mean absolute error for loss
                  optimizer='Adam',    # Adam optimizer
                  metrics=['acc'])     # Print accuracy info during training

    # Train the model
    for inputs, targets, batch in load_batches(batch_size):
        print('\n------------------------------------------------------------------------------------')
        print("Training batch #%d" % batch)
        model.fit(inputs, 
                  targets,
                  steps_per_epoch=batch_size,
                  epochs=total_epochs,
                  callbacks=callback_list, # Call the step_decay function
                  verbose=1)
    
    # Save model
    model.save('model.h5') 
    print('Saved model -> model.h5')
    del model 

# Run the program
main()


In [0]:
from google.colab import files
files.download('model.h5')

In [0]:
from keras.models import load_model
import imageio

DIR = 'dataset/Sony/short/'

INPUTS=[
    '00182_03_0.04s.ARW',
    '20211_01_0.033s.ARW',
    '00186_01_0.033s.ARW',
    '00222_05_0.04s.ARW',
    '00209_00_0.033s.ARW',
    '00232_03_0.04s.ARW', 
    '20201_03_0.04s.ARW',
    '10054_01_0.04s.ARW',
    '00207_01_0.04s.ARW',
    '00215_00_0.033s.ARW'
]

OUT_FMT='test_%s.jpg'

# Loads the saved model
model = load_model('model.h5')

for image in INPUTS:
    iid = image[0:5]
    raw = rawpy.imread(DIR+image)

    i = np.expand_dims(pack_raw(raw), axis=0) * 200
    i = np.minimum(i, 1.0)

    # Runs the model on a new input image
    output = model.predict(i)

    output = np.minimum(np.maximum(output, 0), 1)
    output = output[0, :, :, :]

    output *= 255

    imageio.imwrite(OUT_FMT % (iid), np.array(output, dtype=np.uint8))
    print('saved %s' % (OUT_FMT % (iid)))

    files.download(DIR+image)
    files.download(OUT_FMT % (iid))
    